In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import json
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from modeling_contrastive import LlamaModelEmbedding

[2023-12-03 14:56:43,913] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
model = LlamaModelEmbedding.from_pretrained('checkpoint-84900',
                                 use_flash_attention_2 = True, torch_dtype = torch.bfloat16)

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [4]:
_ = model.cuda()

In [5]:
tokenizer = AutoTokenizer.from_pretrained('checkpoint-84900')

In [6]:
contexts, questions = [], []

with open('/home/ubuntu/lom-agc/ultrachat-lom-agc.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            contexts.append(l[0]['content'].strip())
            questions.append(l[1]['content_ms'].strip())
            
len(contexts), len(questions)

(8034, 8034)

In [7]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    input_ids = tokenizer([contexts[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    contexts_v.append(v)

100%|██████████| 8034/8034 [02:20<00:00, 57.03it/s]


In [8]:
questions_v = []
for i in tqdm(range(len(contexts_v))):
    input_ids = tokenizer([questions[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    questions_v.append(v)

100%|██████████| 8034/8034 [00:58<00:00, 138.08it/s]


In [9]:
contexts_v_np = np.array(contexts_v)[:,0]
questions_v_np = np.array(questions_v)[:,0]

In [10]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 8034/8034 [03:24<00:00, 39.21it/s]


In [11]:
np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1][:10]

array([5249,  959, 5247, 5248, 3470, 2312, 1749, 3585, 1609,  953])

In [12]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.05414488424197162
3 0.11600697037590242
5 0.16143888473985563
10 0.23823749066467514
